In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
from mplsoccer.pitch import Pitch

We read the tracking data for the game between Barcelona and Valladolid on December 22, 2020.

![Cover](../assets/cover.jpg)

In [ ]:
#Let us read the data from the game
df= pd.read_csv('../data/raw/valladolidA.csv')

In [ ]:
df.head(3)

In [ ]:
#We look at Barcelona first
dfBar= df[df['teamId']== 'Barcelona']
dfBar.head(2)
#  or 'BallRecovery' or 'BallTouch' or 'Save' or 'ShotOnPost' or 'Goal'

In [ ]:
dfBarAct= dfBar[dfBar['playerId']< 40][(dfBar['type'] =='Pass') | (dfBar['type'] =='Goal') | (dfBar['type'] =='BallRecovery') \
                | (dfBar['type'] =='BallTouch') | (dfBar['type'] =='Save') | (dfBar['type'] =='ShotOnPost')  ] 

Lets look at player performences 

In [ ]:
dfBarAct.hist('type', by= 'playerId', figsize=(15, 22))


$\textbf{Let us look at Messi's performence:}$


Note that we eliminated his passes

In [ ]:
plt.figure(figsize=(14, 6))
plt.title('Messi')
dfBarwoPass= dfBar[dfBar['type']!='Pass']
dfBarwoPass[dfBarwoPass['playerId']==9]['type'].hist()


$\textbf{We need to add a column to specify player who passed and the player who received the pass.}$

-We also consider successful passes.

-We work with a new data frame called passes

In [ ]:
dfBar['passer']= dfBar['playerId']
dfBar['recipient']=dfBar['playerId'].shift(-1)
passes= dfBar[dfBar['type']== 'Pass']
successful= passes[passes['outcome']== 'Successful']

In [ ]:
# Let us check for the first substitution
df[df['type']=='SubstitutionOff']
# We find that first substitution for Barca was on 70th minute

In [ ]:
# Lets filter for all sucessful passses before the first substitution
successful= successful[successful['minute']<70]

$\textbf{We make a new data frame that look at average position of each player of Barca when they make a pass.}$ 

$\textbf{Also we consider the number of passes from that player.}$

In [ ]:
Av_Loc=successful.groupby('passer').agg({ 'x':'mean', 'y': ['mean', 'count']})
Av_Loc= Av_Loc.reset_index()
Av_Loc.columns=['passer',  'x_mean', 'y_mean', 'y_count']
Av_Loc


In [ ]:
passes_between= successful.groupby(['passer', 'recipient']).type.count().reset_index()
passes_between.rename({'type': 'pass_counts'}, axis='columns', inplace= True)
PBN=passes_between[(passes_between['passer']<50) &  (passes_between['recipient']<50)]
passes_between

In [ ]:
Av_Loc[Av_Loc['passer']==2.0]['x_mean']


In [ ]:
def rec_loc_x(n):
    return Av_Loc[Av_Loc['passer']==n]['x_mean'].values[0]
def rec_loc_y(n):
    return Av_Loc[Av_Loc['passer']==n]['y_mean'].values[0]
Av_Loc.head()

In [ ]:
Mergtwo= passes_between.merge(Av_Loc.drop('y_count', axis=1))
Mergtwo.head(3)

In [ ]:
Mergtwo['rec_loc_x']= Mergtwo['recipient'].apply(rec_loc_x)

In [ ]:
Mergtwo['rec_loc_y']= Mergtwo['recipient'].apply(rec_loc_y)
Mergtwo

In [ ]:
from mplsoccer import VerticalPitch 
pitch = Pitch(pitch_color='grass', line_color='white', stripe=True, constrained_layout=True, goal_type='box')


In [ ]:
fig, ax = pitch.draw(figsize=(9, 8))
pitch.arrows(1.2*Mergtwo['x_mean'], 0.8*Mergtwo['y_mean'], 1.2*Mergtwo['rec_loc_x'], 0.8*Mergtwo['rec_loc_y'], \
             alpha=1, color='black', zorder=1, lw=5,  ax=ax)
pitch.scatter(1.2*Av_Loc['x_mean'], 0.8*Av_Loc['y_mean'], s = 350,  color = 'white', \
               edgecolors='black', linewidth=1, alpha = 1, ax = ax)

plt.title("Pass network for Barcelona", size = 20)                   
for index, row in Mergtwo.iterrows():
    pitch.annotate(row['passer'], xy=(1.2*row['x_mean'], 0.8*row['y_mean']), c ='black', va = 'center', ha = 'center', size = 10, ax = ax)


$$\textbf{Now we will do the same for Valladolid. Note that they lost by 3 goeals. }$$

In [ ]:
dfVal= df[df['teamId']== 'Valladolid']
dfVal.head(2)

Valladolid playerIds are incorrect so we redfine them to make more sense:

In [ ]:
Dic= {121734.0 :   1,
121544.0  :  2,
365371.0  :  3,
279711.0  :  4,
91254.0   :  5,
106868.0  :  6,
362276.0  :  7,
362355.0  :  8,
395579.0  :  9,
369557.0  :  10,
108176.0  :  11,
141533.0  :  12,
382939.0  :  13,
25714.0   :  14,
402185.0  :  15,
323135.0  :  16}

In [ ]:
dfVal['playerId']= dfVal['playerId'].map(Dic)

In [ ]:
# Let us just consider PlayerId smaller than 11
dfVal = dfVal[dfVal['playerId']<12]
dfVal['playerId'].value_counts()

In [ ]:
dfVal['passer']= dfVal['playerId']
dfVal['recipient']=dfVal['playerId'].shift(-1)
passesVal= dfVal[dfVal['type']== 'Pass']
successVal= passesVal[passesVal['outcome']== 'Successful']
successVal

In [ ]:
AV_locVal=successVal.groupby('passer').agg({ 'x':'mean', 'y': ['mean', 'count']})
AV_locVal= AV_locVal.reset_index()
AV_locVal.columns=['passer',  'x_mean', 'y_mean', 'y_count']


In [ ]:
# Let us look at paasess between 
passes_between_Val= successVal.groupby(['passer', 'recipient']).type.count().reset_index()
passes_between_Val.rename({'type': 'pass_counts'}, axis='columns', inplace= True)
passes_between_Val.head(2)

In [ ]:
Mergtwo_Val= passes_between_Val.merge(AV_locVal.drop('y_count', axis=1))
Mergtwo_Val

In [ ]:
def rec_loc_x(n):
    return AV_locVal[AV_locVal['passer']==n]['x_mean'].values[0]
def rec_loc_y(n):
    return AV_locVal[AV_locVal['passer']==n]['y_mean'].values[0]

Mergtwo_Val['rec_loc_x']= Mergtwo_Val['recipient'].apply(rec_loc_x)
Mergtwo_Val['rec_loc_y']= Mergtwo_Val['recipient'].apply(rec_loc_y)
Mergtwo_Val.head(3)

In [ ]:
fig, ax = pitch.draw(figsize=(9, 8))
pitch.arrows(1.2*Mergtwo_Val['x_mean'], 0.8*Mergtwo_Val['y_mean'], 1.2*Mergtwo_Val['rec_loc_x'], 0.8*Mergtwo_Val['rec_loc_y'], \
             alpha=1, color='blue', zorder=1, lw=5,  ax=ax)
pitch.scatter(1.2*AV_locVal['x_mean'], 0.8*AV_locVal['y_mean'], s = 350,  color = 'white', \
               edgecolors='red', linewidth=1, alpha = 1, ax = ax)

plt.title("Pass network for Valladolid", size = 20)                   
for index, row in Mergtwo_Val.iterrows():
    pitch.annotate(row['passer'], xy=(1.2*row['x_mean'], 0.8*row['y_mean']), c ='black', va = 'center', ha = 'center', size = 10, ax = ax)


In [ ]:
df[df['type']== 'Goal']

In [ ]:
df.loc[368:370]

In [ ]:
fig, ax = pitch.draw(figsize=(9, 8))
pitch.arrows(df.loc[368:370]['x']*1.2, df.loc[368:370]['y']*0.8, df.loc[368:370]['endX']*1.2, df.loc[368:370]['endX']*0.8, \
             alpha=0.7, color='black', zorder=1, lw=3,  ax=ax)

pitch.scatter(df.loc[368:370]['x']*1.2, df.loc[368:370]['x']*0.8, s = 150,  color = 'blue', \
               edgecolors='black', linewidth=1, alpha = 1, ax = ax)

